In [ ]:
import pandas as pd

#-----------------------------
#Wildfire Count By Year
#-----------------------------

df = pd.read_csv('../Data/PreprocessedData/CA_Fire_Perimeter_FILTERED.csv')

series_Years = df.value_counts("Year")

df_YearCount = pd.DataFrame({'Year':series_Years.index, 'Count':series_Years.values})

df_YearCount = df_YearCount.sort_values(by = 'Year')

df_YearCount = df_YearCount.reset_index()

df_YearCount = df_YearCount.drop(columns=["index"])

df_YearCount = df_YearCount.astype(int)


df_YearCount.to_csv('../Data/Output/Tables/WildfireCountByYear.csv', index=False)

print(df_YearCount)

plt.plot(df_YearCount['Year'], df_YearCount['Count'])
plt.xlabel('Year')
plt.ylabel('Occurence')
plt.xticks(ticks=plt.xticks()[0], labels=plt.xticks()[0].astype(int))
plt.axis((df_YearCount['Year'].min(), df_YearCount['Year'].max(), df_YearCount['Count'].min(), df_YearCount['Count'].max())) 
ax.set_xticks((df_YearCount['Year'].min(), df_YearCount['Year'].max()+1, 24), labels=(df_YearCount['Year'].min(), df_YearCount['Year'].max()+1, 24))


plt.savefig('../Data/Output/Plots/OccurenceByYear.png', bbox_inches="tight")

plt.show()




In [ ]:
from matplotlib import pyplot as plt

#-----------------------------
#Wildfire Count By Cause
#-----------------------------

def getCause(causeNum):
    return causeDict[causeNum]


series_Cause = df.value_counts("Cause")

df_Cause = pd.DataFrame({'Cause':series_Cause.index, 'Count':series_Cause.values})

df_Cause = df_Cause.sort_values(by = 'Cause')

df_Cause = df_Cause.reset_index()

df_Cause['Cause'] = df_Cause['Cause'].apply(getCause)

df_Cause = df_Cause.drop(columns=["index"])

df_Cause = df_Cause.sort_values(by=['Count'])

bars_heights = df_Cause['Count']
bars_label = df_Cause['Cause']

plt.bar(range(len(bars_label)), bars_heights)
plt.xticks(range(len(bars_label)), bars_label, rotation='vertical')
plt.xlabel('Cause')
plt.ylabel('Number')

plt.savefig('../Data/Output/Plots/NumberByCause.png', bbox_inches="tight")

plt.show()

In [ ]:
import pandas as pd

#-----------------------------
#Wildfire Count By Year and By Cause Subsets
#-----------------------------

causeDict = {
            1: 'Lightning' ,
            2 : 'Equipment Use' ,
            3 : 'Smoking' ,
            4 : 'Campfire' ,
            5 : 'Debris' ,
            6 : 'Railroad', 
            7 : 'Arson' ,
            8 : 'Playing with fire' ,
            9 : 'Miscellaneous' ,
            10 : 'Vehicle' ,
            11 : 'Powerline', 
            12 : 'Firefighter Training', 
            13 : 'Non-Firefighter Training' ,
            14 : 'Unknown/Unidentified' ,
            15 : 'Structure' ,
            16 : 'Aircraft' ,
            17 : 'Volcanic' ,
            18 : 'Escaped Prescribed Burn' ,
            19 : 'Illegal Alien Campfire'
            }

#returns string value of cause integer
def getCause(causeNum):
    return causeDict[causeNum]

df = pd.read_csv('../Data/PreprocessedData/CA_Fire_Perimeter_FILTERED.csv')


df['CauseName'] = df['Cause'].apply(getCause)


df_data = pd.DataFrame(columns=['Year', 'Electric systems', 'Campers', 'Other recorded causes', 'Known', 'All'])

for year in range(2004,2024):
    powerlineNum = df[(df.CauseName=="Powerline") & (df.Year==year)].shape[0]
    campNum = df[(df.CauseName=="Campfire") & (df.Year==year)].shape[0]
    otherNum = df[(df.CauseName != "Powerline") & (df.CauseName != "Campfire") 
                  & (df.CauseName != "Unknown/Unidentified") & (df.Year==year)].shape[0]
    knownNum = df[(df.CauseName != "Unknown/Unidentified") & (df.Year==year)].shape[0]
    allNum = df[(df.Year==year)].shape[0]
        
    df_data = pd.concat([pd.DataFrame([[year, powerlineNum, campNum, otherNum, knownNum, allNum]], columns=df_data.columns), df_data], ignore_index=True)

    
df_data.to_csv('../Data/Output/Tables/WildfireCountByCause.csv', index=False)

#-----------------------------
#Summary Statistic Generation
#-----------------------------

#assumption being that it's the total number of fires from 2004-2024

series_Sum = df_data.sum(axis=0)
df_Sum = pd.DataFrame({'Type':series_Sum.index, 'Sum':series_Sum.values})
df_Sum = df_Sum.drop([0])

allNum = df_Sum.loc[df_Sum.Type=='All','Sum'].values[0]

df_Sum = df_Sum.drop(df_Sum[df_Sum.Type == 'All'].index)

df_Sum['Percent'] = 100 * df_Sum['Sum']/allNum

powerlineArea = df[(df.CauseName=="Powerline")].mean()['Size']
campArea = df[(df.CauseName=="Campfire")].mean()['Size']
otherArea = df[(df.CauseName != "Powerline") & (df.CauseName != "Campfire") 
              & (df.CauseName != "Unknown/Unidentified")].mean()['Size']

knownArea = df[(df.CauseName != "Unknown/Unidentified")].mean()['Size']

df_Sum.insert(len(df_Sum.columns), "Average Area", [powerlineArea, campArea, otherArea, knownArea], True)


print(df_Sum)

df_Sum.to_csv('../Data/Output/Tables/WildfireCauseStats.csv', index=False)

In [ ]:
#-----------------------------
#Wildfire Area by Cause 
#-----------------------------

areaList = []
causeList = []

diction = {}

for x in range(1,20):
    Area = df[(df.CauseName==causeDict[x])].mean()['Size']
    areaList.append(Area)
    causeList.append(causeDict[x])
    diction[causeDict[x]] = Area

dSort = dict(sorted(diction.items(), key=lambda item: item[1]))
#print(type(dSort))
bars_heights = dSort.values()
bars_label = dSort.keys()


plt.bar(range(len(bars_label)), bars_heights)
plt.xticks(range(len(bars_label)), bars_label, rotation='vertical')
plt.xlabel('Cause')
plt.ylabel('Area (acres)')

plt.savefig('../Data/Output/Plots/AverageAreaByCause.png', bbox_inches="tight")

plt.show()